In [1]:
%matplotlib widget

In [1]:
from genSTEM import Model
import matplotlib.pyplot as plt
import numpy as np
import cupy as cp
from importlib import reload
reload(Model)
from tqdm.auto import tqdm

In [2]:
from ase.io import read
from ase.build import make_supercell

atoms = read("0013687130_v6bxv2_tv0.1bxv0.0_d1.8z_traj.xyz")
atoms = make_supercell(atoms, np.diag((1,2,1)))
mask = atoms.positions[:,2] > 35
del atoms[mask]
atoms.rotate([1,0,0], (0,0,1))

from skimage.transform import warp
from matplotlib.transforms import Affine2D

atoms[1716].number = 60
atoms[1766].number = 60

In [3]:
reload(Model)


<module 'genSTEM.Model' from 'c:\\users\\thomasaar\\documents\\github\\generate_scan_distorted_stem\\genstem\\genSTEM\\Model.py'>

In [7]:
nImages = 18
pixel_size = 0.3
drift_strength = 5 # approx 10 -> becomes 10 / np.prod(image_shape)
jitter = 0.5
vacuum = 0.
#drift_vector = [1,1]
random_angle = np.random.random() * 2*np.pi
drift_vector = [np.cos(random_angle),np.sin(random_angle)]
centre_drift = False

In [8]:
%%time
reload(Model)
images = []

# m0 = Model.ImageModel(atoms, pixel_size=pixel_size, vacuum=vacuum,
#                 scan_rotation=0, centre_drift=centre_drift)
# img0 = m0.generate_cupy()
# side = np.minimum(*img0.shape) # Use to make image square

scanangles = cp.linspace(0, 360, nImages, endpoint=False)
for scanangle in tqdm(scanangles):
    m = Model.ImageModel(atoms, scan_rotation=scanangle,
                   pixel_size=pixel_size, vacuum=vacuum,
                    drift_strength=drift_strength, 
                    drift_vector=drift_vector,
                    jitter_strength=jitter, 
                    centre_drift=centre_drift,
                    fast=True,
                  )
    img = m.generate_cupy()
    side = np.minimum(*img.shape)
    images.append(img[:side, :side])
images = cp.stack(images)
print(f"Size: {images.nbytes / 1e9} GB")
print(f"Shape: {images.shape}")


Size: 0.001937664 GB
Shape: (18, 116, 116)
Wall time: 1.16 s


In [9]:
# cp.save("36img251.npy", images)
# cp.save("36img251_angles.npy", scanangles)

In [31]:
# images = cp.load("9img6k.npy")
# scanangles = cp.load("9img6k_angles.npy")
# print(f"Size: {images.nbytes / 1e9} GB")

In [130]:
scans

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29])

In [ ]:
np.rot90()

In [131]:
import hyperspy.api as hs
s = hs.load(r"C:\Users\thomasaar\Documents\data\2\*", stack=True)

images = cp.array(s.data).astype(float)
scans = cp.arange(len(images))
scanangles = scans*90
images = cp.stack([cp.rot90(img, scan, axes=(-2,-1)) for img, scan in zip(images, scans)])

images = images[:,128:-128, 128:-128]
#images = cp.pad(images, ((0,0), (256, 256), (256,256)))
images[images == 0] = np.mean(images)

In [119]:
# img1, img2 = images[0], images[-1]
# s, m =  Model.hybrid_correlation(img1, img2, normalize_mean=True)

# IM = img1 + Model.translate(img2, (s[0], s[1]))

In [120]:
# scanangles = scanangles[:2]
# images = images[:2]

In [132]:
fig, AX = plt.subplots(6, 6, figsize=(10,10))
for ax, img in zip(AX.flatten(), images):
    img = img.get()
    ax.imshow(img)
    ax.axis('off')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [133]:
%matplotlib widget

In [142]:
from IPython.display import HTML
display(HTML("""
<style>
.jp-OutputArea-prompt:empty {
  padding: 0;
  border: 0;
}
</style>
"""))

In [145]:
fig = plt.figure(figsize = (9,3))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
reload(Model)
print(f"\nActual angle: {np.round(np.rad2deg(random_angle), 2)}°")
best_angle, best_str = Model.estimate_drift(images, scanangles, tolerancy_percent=1, debug=True)
print(f"Missed by {np.round(best_angle - np.rad2deg(random_angle),2)}°")


Actual angle: 116.18°

Iteration #0: Best guess: Angle: 0°, Strength: 0.00e+00
Limits: [0.0°, 337.5°], [0.00e+00, 1.30e-03]


In [126]:
reload(Model)
mean = Model.create_final_stack_and_average(images, scanangles, best_str, best_angle)
plt.figure()
plt.imshow(mean)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [127]:
warped_images = [Model.warp_image_cuda(img, scan, best_str, best_angle) for img, scan in zip(images, scanangles)]
fig, AX = plt.subplots(6, 6, figsize=(10,10))
for ax, img in zip(AX.flatten(), warped_images):
    img = img.get()
    ax.imshow(img, vmin=5e5)#[50:-50, 50:-50])
    ax.axis('off')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [91]:
no_warp_images = cp.array(s.data.astype(float))
img1 = no_warp_images[0]
no_warp_mean = img1.copy()
for img in no_warp_images[1:]:
    s, m =  Model.hybrid_correlation(img1, img, normalize_mean=True)
    no_warp_mean += Model.translate(img, (s[0], s[1]))

In [92]:
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(6,3))
ax1.imshow(mean[256:-256, 256:-256])
ax2.imshow(no_warp_mean[256:-256, 256:-256].get())

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [151]:
# fig, AX = plt.subplots(2, 2, figsize=(10,10))
# for ax, img in zip(AX.flatten(), nonan):
#     ax.imshow(img)
#     ax.axis('off')

In [152]:
# fig, AX = plt.subplots(6, 6, figsize=(10,10))
# for ax, img in zip(AX.flatten(), warped_images_nonan):
#     ax.imshow(img.get())
#     ax.axis('off')

In [201]:
str_limits_step = (0, 1/images[0].shape[0], 0.2/images[0].shape[0])

In [202]:
strengths = np.arange(*str_limits_step)

3.0995030722826096

In [218]:
i = 1
strength = strengths[i]
angle = np.rad2deg(random_angle)
scanangle1, scanangle2 = scanangles[0], scanangles[9]
i1 = Model.warp_image_cuda(images[0], scanangle1, strength, angle)
i2 = Model.warp_image_cuda(images[9], scanangle2, strength, angle)

fig, (ax1, ax2) = plt.subplots(ncols=2)
ax1.imshow(i1.get())
ax2.imshow(i2.get())

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [186]:
angle, strength = pairs[i]
i1 = Model.warp_image_cuda(img1, scanangle1, strength, angle)
i2 = Model.warp_image_cuda(img2, scanangle2, strength, angle)
#i3 = Model.warp_image_cuda(img3, scanangle3, strength, angle)

In [187]:
shift = shifts[i][0]
def translate(img, shift):
    return np.roll(img, shift, axis=(0,1))

A = i1.get()
B = translate(i2.get(), (shift[0], shift[1]))
#C = translate(i3.get(), (shift[0], shift[1]))


fig, (ax1, ax2, ax3, ) = plt.subplots(ncols=3, figsize=(9,5))
ax1.imshow(A)
ax2.imshow(B)
#ax3.imshow(C)
ax3.imshow(np.mean(np.asarray([A,B]), axis=0))


TypeError: 'int' object is not subscriptable

In [148]:
A,B,C = (img.get() for img in images[:3])
fig, (ax1, ax2, ax3, ax4) = plt.subplots(ncols=4, figsize=(12,10))
ax1.imshow(A)
ax2.imshow(B)
ax3.imshow(C)
ax4.imshow(np.mean(np.asarray([A,B,C]), axis=0))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …